# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geopandas as gpd
import os
import gmaps
import geoviews as gv


# Import API key
from api_keys import geoapify_key
from api_keys import gmap_key


In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,12.12,92,82,1.34,NZ,1738698442
1,1,wailua homesteads,22.0669,-159.3780,21.44,77,20,4.63,US,1738698443
2,2,grytviken,-54.2811,-36.5092,5.20,68,25,10.81,GS,1738698444
3,3,blackmans bay,-43.0167,147.3167,12.07,75,40,2.36,AU,1738698446
4,4,iqaluit,63.7506,-68.5145,-26.15,64,20,4.63,CA,1738698448


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha=.75
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_ideal = city_data_df.loc[(city_data_df["Max Temp"]>=15) & (city_data_df["Max Temp"]<=35) & (city_data_df)["Humidity"]<=60 & (city_data_df["Cloudiness"]<=60)]

# Drop any rows with null values
city_clean = city_ideal.dropna()

# Display sample data
city_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,12.12,92,82,1.34,NZ,1738698442
2,2,grytviken,-54.2811,-36.5092,5.20,68,25,10.81,GS,1738698444
3,3,blackmans bay,-43.0167,147.3167,12.07,75,40,2.36,AU,1738698446
4,4,iqaluit,63.7506,-68.5145,-26.15,64,20,4.63,CA,1738698448
5,5,invercargill,-46.4000,168.3500,11.68,95,89,0.89,NZ,1738698449
...,...,...,...,...,...,...,...,...,...,...
568,568,north bend,43.4065,-124.2243,6.26,81,20,1.54,US,1738699150
569,569,howard springs,-12.4970,131.0470,24.95,90,100,5.23,AU,1738699151
572,572,bridgewater,44.3835,-64.5155,-0.18,68,60,6.17,CA,1738698999
573,573,jayapura,-2.5333,140.7000,23.66,94,100,1.76,ID,1738699155


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_clean[["Lat", "Lng", "City", "Country", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\carls\AppData\Local\Temp\ipykernel_28256\254656205.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,Lat,Lng,City,Country,Humidity,Hotel Name
0,-43.9535,-176.5597,waitangi,NZ,92,
2,-54.2811,-36.5092,grytviken,GS,68,
3,-43.0167,147.3167,blackmans bay,AU,75,
4,63.7506,-68.5145,iqaluit,CA,64,
5,-46.4000,168.3500,invercargill,NZ,95,
...,...,...,...,...,...,...
568,43.4065,-124.2243,north bend,US,81,
569,-12.4970,131.0470,howard springs,AU,90,
572,44.3835,-64.5155,bridgewater,CA,68,
573,-2.5333,140.7000,jayapura,ID,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

Had a lot of issues with geoapify keys, upon some research a lot of people were saying it did not work for them and they ended up using google maps, which is what I did.

In [27]:

hotel_data = []

for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    city = row["City"]

    print(f"Retrieving Results for {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=10000&type=lodging&key={gmap_key}").json()
    results = response.get("results", [])

    try:
        if results:  # Check if there are any results
            hotel_name = results[0]["name"]  # Get the first hotel name
            print(f"The Best and Nearest hotel to {city} at {latitude}, {longitude} is {hotel_name}")
            hotel_df.loc[index, "Hotel Name"] = hotel_name
        else:
            print("No results found.")
            hotel_df.loc[index, "Hotel Name"] = "No Hotel Found"

    except (KeyError, IndexError) as e:
        print("No result...moving on")
        hotel_df.loc[index, "Hotel Name"] = "No Hotel Found"

# Optionally, display the updated DataFrame
print(hotel_df)

Retrieving Results for waitangi.
The Best and Nearest hotel to waitangi at -43.9535, -176.5597 is Hotel Chatham
Retrieving Results for grytviken.
The Best and Nearest hotel to grytviken at -54.2811, -36.5092 is The Salem Inn
Retrieving Results for blackmans bay.
The Best and Nearest hotel to blackmans bay at -43.0167, 147.3167 is Kingston Hotel
Retrieving Results for iqaluit.
The Best and Nearest hotel to iqaluit at 63.7506, -68.5145 is Aqsarniit Hotel & Conference Centre
Retrieving Results for invercargill.
The Best and Nearest hotel to invercargill at -46.4, 168.35 is Beach Road Holiday Park
Retrieving Results for bethel.
The Best and Nearest hotel to bethel at 41.3712, -73.414 is Ethan Allen Hotel
Retrieving Results for vorkuta.
The Best and Nearest hotel to vorkuta at 67.5, 64.0 is Sever
Retrieving Results for funafuti.
The Best and Nearest hotel to funafuti at -8.5243, 179.1942 is Funafuti Lagoon Hotel
Retrieving Results for severo-yeniseyskiy.
The Best and Nearest hotel to severo

In [28]:
hotel_df

,Lat,Lng,City,Country,Humidity,Hotel Name
0,-43.9535,-176.5597,waitangi,NZ,92,Hotel Chatham
2,-54.2811,-36.5092,grytviken,GS,68,The Salem Inn
3,-43.0167,147.3167,blackmans bay,AU,75,Kingston Hotel
4,63.7506,-68.5145,iqaluit,CA,64,Aqsarniit Hotel & Conference Centre
5,-46.4000,168.3500,invercargill,NZ,95,Beach Road Holiday Park
...,...,...,...,...,...,...
568,43.4065,-124.2243,north bend,US,81,Itty Bitty Inn
569,-12.4970,131.0470,howard springs,AU,90,BIG4 Howard Springs Holiday Park
572,44.3835,-64.5155,bridgewater,CA,68,Best Western Plus Bridgewater Hotel & Conventi...
573,-2.5333,140.7000,jayapura,ID,94,RedDoorz near Pelabuhan Jayapura


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 500,
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)


# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)